In [7]:
import pandas as pd
import numpy as np

In [95]:
df = pd.read_excel('../datasrc/HIST_PAINEL_COVIDBR_22mai2020.xlsx')

In [96]:
df.head(5)

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,0,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,0,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-01,10,210147125.0,2,0,NaN,NaN


In [97]:
df.columns

Index(['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude',
       'nomeRegiaoSaude', 'data', 'semanaEpi', 'populacaoTCU2019',
       'casosAcumulado', 'obitosAcumulado', 'Recuperadosnovos',
       'emAcompanhamentoNovos'],
      dtype='object')

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103202 entries, 0 to 103201
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   regiao                 103202 non-null  object 
 1   estado                 103115 non-null  object 
 2   municipio              10692 non-null   object 
 3   coduf                  103202 non-null  int64  
 4   codmun                 101219 non-null  float64
 5   codRegiaoSaude         10692 non-null   float64
 6   nomeRegiaoSaude        10692 non-null   object 
 7   data                   103202 non-null  object 
 8   semanaEpi              103202 non-null  int64  
 9   populacaoTCU2019       12675 non-null   float64
 10  casosAcumulado         103202 non-null  int64  
 11  obitosAcumulado        103202 non-null  int64  
 12  Recuperadosnovos       34 non-null      float64
 13  emAcompanhamentoNovos  34 non-null      float64
dtypes: float64(5), int64(4), object(5)
m

In [99]:
df['estado'].unique()

array([nan, 'RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE',
       'RN', 'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR',
       'SC', 'RS', 'MS', 'MT', 'GO', 'DF'], dtype=object)

## Base de dados de longitude e latitude

In [100]:
df_localizacoes = pd.read_csv('../datasets/datasets_localizacao_municipios.csv')

In [101]:
df_localizacoes.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


In [102]:
df_localizacoes['codmun'] = df_localizacoes['codigo_ibge']//10
df_localizacoes = df_localizacoes[['codmun', 'latitude', 'longitude']]
df_localizacoes.head()

,codmun,latitude,longitude
0,520005,-16.75730,-49.4412
1,310010,-18.48310,-47.3916
2,520010,-16.19700,-48.7057
3,310020,-19.15510,-45.4444
4,150010,-1.72183,-48.8788


## Divide as base de dados (cidades e estados)

Primeiramente vamos extrair a base apenas de municípios.

In [103]:
df_city = df[df['municipio'].notnull()]

In [104]:
df_city = df_city.fillna(0)

In [105]:
df_city = pd.merge(df_city, df_localizacoes, on='codmun')

estados = {'Acre': 'AC',
           'Alagoas': 'AL',
           'Amapá': 'AP',
           'Amazonas': 'AM',
           'Bahia': 'BA',
           'Ceará': 'CE',
           'Distrito Federal': 'DF',
           'Espírito Santo': 'ES',
           'Goiás': 'GO',
           'Maranhão': 'MA',
           'Mato Grosso': 'MT',
           'Mato Grosso do Sul': 'MS',
           'Minas Gerais': 'MG',
           'Pará': 'PA',
           'Paraíba': 'PB',
           'Paraná': 'PR',
           'Pernambuco': 'PE',
           'Piauí': 'PI',
           'Rio de Janeiro': 'RJ',
           'Rio Grande do Norte': 'RN',
           'Rio Grande do Sul': 'RS',
           'Rondônia': 'RO',
           'Roraima': 'RR',
           'Santa Catarina': 'SC',
           'São Paulo': 'SP',
           'Sergipe': 'SE',
           'Tocantins': 'TO'} 

df_estados_siglas = pd.DataFrame(list(estados.items()), columns='uf estado'.split())
df_city = pd.merge(df_city, df_estados_siglas, on='estado')

In [106]:
df_city.head(5)

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos,latitude,longitude,uf
0,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,Zona da Mata,2020-05-20,21,22945.0,2,0,0.0,0.0,-11.92830,-61.9953,Rondônia
1,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,Zona da Mata,2020-05-21,21,22945.0,2,0,0.0,0.0,-11.92830,-61.9953,Rondônia
2,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,Zona da Mata,2020-05-22,21,22945.0,2,0,0.0,0.0,-11.92830,-61.9953,Rondônia
3,Norte,RO,Ariquemes,11,110002.0,11001.0,Vale do Jamari,2020-05-20,21,107863.0,191,2,0.0,0.0,-9.90571,-63.0325,Rondônia
4,Norte,RO,Ariquemes,11,110002.0,11001.0,Vale do Jamari,2020-05-21,21,107863.0,205,2,0.0,0.0,-9.90571,-63.0325,Rondônia


Agora vamos extrair a base de estados apenas com as colunas de interesse.

In [107]:
df_state = df[(df['municipio'].isnull()) & (df['estado'].notnull())]

In [108]:
df_state = df_state.drop(['municipio', 
                          'codmun', 
                          'codRegiaoSaude', 
                          'nomeRegiaoSaude', 
                          'emAcompanhamentoNovos'], axis=1)

df_state = df_state.fillna(0)

In [109]:
df_state.head()

,regiao,estado,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos
87,Norte,RO,11,2020-03-20,12,1777225.0,1,0,0.0
88,Norte,RO,11,2020-03-21,12,1777225.0,1,0,0.0
89,Norte,RO,11,2020-03-22,13,1777225.0,3,0,0.0
90,Norte,RO,11,2020-03-23,13,1777225.0,3,0,0.0
91,Norte,RO,11,2020-03-24,13,1777225.0,3,0,0.0


In [110]:
df_country = df[(df['municipio'].isnull()) & (df['estado'].isnull())]

In [111]:
df_country = df_country.drop(['municipio', 
                              'codmun', 
                              'estado',
                              'codRegiaoSaude', 
                              'nomeRegiaoSaude', 
                              'emAcompanhamentoNovos'], axis=1)

df_country = df_country.fillna(0)
df_country['casosPorDia'] = df_country['casosAcumulado'].diff().fillna(0)
df_country['obitosPorDia'] = df_country['obitosAcumulado'].diff().fillna(0)

In [112]:
df_country.head()

,regiao,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,casosPorDia,obitosPorDia
0,Brasil,76,2020-02-26,9,210147125.0,1,0,0.0,0.0,0.0
1,Brasil,76,2020-02-27,9,210147125.0,1,0,0.0,0.0,0.0
2,Brasil,76,2020-02-28,9,210147125.0,1,0,0.0,0.0,0.0
3,Brasil,76,2020-02-29,9,210147125.0,2,0,0.0,1.0,0.0
4,Brasil,76,2020-03-01,10,210147125.0,2,0,0.0,0.0,0.0


Vamos salvar os dataframes em arquivo excel para leitura no PowerBI.

In [113]:
df_city.to_excel('../datasets/dataset_covid_city.xlsx')

In [114]:
df_state.to_excel('../datasets/dataset_covid_state.xlsx')

In [115]:
df_country.to_excel('../datasets/dataset_covid_country.xlsx')

## Sandbox

In [10]:
from pathlib import Path

def most_recent_file(path):
  # Identifica as datas de modificação
  data_modificacao = lambda f: f.stat().st_mtime

  # Encontra o diretório no qual estão os arquivos
  directory = Path(path)

  # Seleciona apenas os arquivos com extensão .xlsx
  files = directory.glob('*.xlsx')

  # Ordena os arquivos do mais recente para o menos recente
  sorted_files = sorted(files, key=data_modificacao, reverse=True)

  # retorna o arquivo mais recente
  return sorted_files[0]

In [12]:
PATH = '../datasrc/'

df = pd.read_excel(most_recent_file(PATH))

In [15]:
str(most_recent_file(PATH))

'..\\datasrc\\HIST_PAINEL_COVIDBR_24mai2020 (2).xlsx'